# Reinforcement Learning - An Introduction

Richard S. Sutton and Andrew G. Barto

---

## Chapitre 2

Algorithme du **multi armed bandits** version **greedy**.

***Notes : les autres algorithmes sont en cours d'implémentation***

In [ ]:
from typing import Callable, Dict, Tuple

import matplotlib.pyplot as plt
import torch
import torch.distributions as distrib
import torch.nn as nn

# k-armed bandits

### Implémentation du problème de *multi armed bandits*

La découverte intéressante ici est le module de distributions fourni par PyTorch. Super pratique pour générer des distributions aléatoires suivant la loi souhaitée, ou bien jouer avec les fonctions de probabilités sous-jacentes.

In [ ]:
class kBanditGaussian:
    def __init__(self, k: int, *, points_loc: float=0., points_scale: float=1., reward_loc: float=0., reward_scale: float=1.) -> None:
        self.k = k
        self.normal_loc = points_loc
        self.normal_scale = points_scale
        self.reward_loc = reward_loc
        self.reward_scale = reward_scale

        self._normal_point_generator = distrib.normal.Normal(torch.scalar_tensor(points_loc), torch.scalar_tensor(points_scale))
        self._normal_reward_generator = distrib.normal.Normal(torch.scalar_tensor(reward_loc), torch.scalar_tensor(reward_scale))

        self.points = self._points_initialization()
        self.optimal_action = torch.argmax(self.points).item()
    
    def _points_initialization(self) -> Dict[int, float]:
        return self._normal_point_generator.sample(sample_shape=(self.k,))
    
    def get_reward(self, action: int) -> float:
        return self.points[action] + self._normal_reward_generator.sample(sample_shape=(1,)).item()
    
    def __call__(self, action: int) -> float:
        return self.get_reward(action)

In [ ]:
# multi armed bandits
bandit_problem = kBanditGaussian(k=10, points_loc=0., points_scale=1.)

# $\epsilon$-greedy algorithm

### Implémentation de l'algorithme greedy

Plus de détails quand à l'implémentation en PyTorch pour ceux qui ne seraient pas familiers avec le *framework* :

- Principe de base : \
Le *framework* exploite largement l'orienté objet, donc une couche de neurones ou un modèle hérite de la classe `torch.nn.Module`. Afin de définir le traitement de la couche et du modèle en général, il faut définir et implémenter la méthode `forward`.

- La méthode `register_buffer` : \
Dans la classe héritée par `torch.nn.Module`, le *framework* reconnait automatiquement les tenseurs ou module qui sont déclarés comme étant de type (ou héritée de) `torch.nn.Parameter` ou `torch.nn.Module`. Ainsi lorsque nous désirons sauvegarder le modèle, le module génère ce qu'on appelle un *state dict* qui est un dictionnaire des paramètres du modèle, qui sont rattachés aux bons modules. Cependant pas tous les attributs déclarés dans le constructeur (`__init__`) avec la racine `self` sont pris en compte, car n'étant pas déclarés comme paramètres apprenables (`torch.nn.Parameter`). Donc, afin d'enregistrer un quelconque tenseur dans le *state dict* généré, nous pouvons faire appelle à cette méthode `register_buffer`. Une fois cette méthode appelée, l'attribut de classe est ainsi généré et nous pouvons y faire appelle via la racine `self` au sein des méthodes de la classe.

In [ ]:
class GreedyBandit(nn.Module):
    def __init__(self, epsilon: float, k: int) -> None:
        super(GreedyBandit, self).__init__()
        self.register_buffer('epsilon', torch.scalar_tensor(epsilon))
        self.register_buffer('k', torch.scalar_tensor(k))
        self.register_buffer('Q_action_values', torch.zeros(size=(k,), dtype=torch.float))
        self.register_buffer('Q_action_values_count', torch.zeros(size=(k,), dtype=torch.int))

        self._use_greedy_action_selector = distrib.categorical.Categorical(torch.tensor([1 - epsilon, epsilon]))
        self._others_actions_selector = distrib.categorical.Categorical(torch.tensor([1/k for _ in range(k)]))
    
    def _select_action(self) -> int:
        if self._use_greedy_action_selector.sample().item():
            return self._others_actions_selector.sample().item()
        
        return torch.argmax(self.Q_action_values).item()
    
    def _update_q_action_values(self, action: int, reward: float) -> None:
        self.Q_action_values_count[action] = self.Q_action_values_count[action] + 1
        self.Q_action_values[action] = self.Q_action_values[action] + 1 / self.Q_action_values_count[action] * (reward - self.Q_action_values[action]) 
    
    def forward(self, kbandit: Callable) -> Tuple[torch.Tensor, torch.Tensor]:
        action = self._select_action()
        with torch.no_grad():
            reward = kbandit(action)
            self._update_q_action_values(action, reward)

        return (torch.scalar_tensor(action, dtype=torch.int), torch.scalar_tensor(reward, dtype=torch.float))

### Exécution de l'algorithme

L'objectif étant de comprendre notre algorithme et de connaître son évolution. Je vous propose ici de répéter 2000 fois 1000 itérations de l'entrainement, comme proposé dans l'ouvrage.

In [ ]:
# Baseline -> Sans exploration -> epsilon = 0
baseline_histories = []
for _ in range(2000):
    greedy_bandit = GreedyBandit(0., 10)
    history = []
    for _ in range(1000):
        history.append(greedy_bandit(bandit_problem))
    
    baseline_histories.append(history)

baseline_histories = torch.tensor(baseline_histories)

In [ ]:
# epsilon-greedy
greedy_histories = []
for _ in range(2000):
    greedy_bandit = GreedyBandit(0.1, 10)
    history = []
    for _ in range(1000):
        history.append(greedy_bandit(bandit_problem))
    
    greedy_histories.append(history)

greedy_histories = torch.tensor(greedy_histories)

# Optimistic Initial Values

Dans ce cas précis, nous intitialisons nos Q-valeurs à une certaine valeur dans le but d'avoir une meilleure exploration et que toutes les actions puissent être visitées. Dans le cadre du *k-bandits*, cela marchera bien si nous trouvons une valeur supérieure à celle des moyennes des *k* distributions, nous pouvons être sûr que chacune des actions auront été exécutées au moins une fois.

In [ ]:
class OptimisticGreedyBandit(GreedyBandit):
    def __init__(self, initial_value: int, epsilon: float, k: int) -> None:
        super(OptimisticGreedyBandit, self).__init__(epsilon, k)

        self.Q_action_values = torch.ones(size=(k,), dtype=torch.float) * initial_value

In [ ]:
# optimiscal initial values
optimistic_greedy_histories = []
for _ in range(2000):
    optimistic_greedy_bandit = OptimisticGreedyBandit(5., 0., 10)
    history = []
    for _ in range(1000):
        history.append(optimistic_greedy_bandit(bandit_problem))
    
    optimistic_greedy_histories.append(history)

optimistic_greedy_histories = torch.tensor(optimistic_greedy_histories)

# Upper-Confidence-Bound Action selection

L'objectif dans le cadre de l'*Upper-Confidence-Bound* (ou UCB) est de s'affranchir d'une probabilité d'exploration et de forcer l'exploration par le nombre de fois que l'action a été exécutée. Plus nous itérons, plus l'incertitude sur la valeur de récompense d'une action est levée. Afin d'avoir une meilleure intuition, voici la formule de séléction d'une action via UCB :

$A_t = \textrm{argmax}_a \left[ Q_t(a) + c \sqrt{\frac{ln(t)}{N_t(a)}} \right]$, avec $t$ représentant l'itération actuelle, et $N_t(a)$ le nombre de fois que l'action $a$ a été exécutée jusqu'à la $t$ ième itération.

In [ ]:
class UpperConfidenceBound(nn.Module):
    def __init__(self, c: float, k: int, eps: float=1e-7) -> None:
        super(UpperConfidenceBound, self).__init__()

        self.register_buffer('c', torch.scalar_tensor(c, dtype=torch.float))
        self.register_buffer('k', torch.scalar_tensor(k, dtype=torch.int))
        self.register_buffer('eps', torch.scalar_tensor(eps, dtype=torch.float))
        self.register_buffer('Q_action_values', torch.zeros(size=(k,), dtype=torch.float))
        self.register_buffer('Q_action_values_count', torch.zeros(size=(k,), dtype=torch.int))
    
    def _select_action(self, t: int) -> int:
        current_values = self.Q_action_values + self.c * torch.sqrt(torch.log(torch.scalar_tensor(t)) / (self.Q_action_values_count + self.eps))
        equalities = torch.where(current_values == current_values.max())[0]

        if equalities.shape[0] > 1:
           selection = distrib.categorical.Categorical(torch.tensor([1 / equalities.shape[0] for _ in range(equalities.shape[0])])).sample().item()
           return equalities[selection]

        return torch.argmax(self.Q_action_values + self.c * torch.sqrt(torch.log(torch.scalar_tensor(t)) / (self.Q_action_values_count + self.eps)))
    
    def _update_q_action_values(self, action: int, reward: float) -> None:
        self.Q_action_values_count[action] = self.Q_action_values_count[action] + 1
        self.Q_action_values[action] = self.Q_action_values[action] + 1 / self.Q_action_values_count[action] * (reward - self.Q_action_values[action])

    def forward(self, kbandit: Callable,  t: int) -> Tuple[torch.Tensor, torch.Tensor]:
        action = self._select_action(t)
        with torch.no_grad():
            reward = kbandit(action)
            self._update_q_action_values(action, reward)

        return (torch.scalar_tensor(action, dtype=torch.int), torch.scalar_tensor(reward, dtype=torch.float))

In [ ]:
# Upper Confidence Bound
ucb_greedy_histories = []
for _ in range(2000):
    ucb_action_selection = UpperConfidenceBound(2., 10)
    history = []
    for i in range(1000):
        history.append(ucb_action_selection(bandit_problem, i+1))
    
    ucb_greedy_histories.append(history)

ucb_greedy_histories = torch.tensor(ucb_greedy_histories)

# Gradient Bandits Algorithms

Dans ce dernier cas, nous utilisons nous plus une valeur Q, mais une valeur de préférence. Cette préférence est déterminée par la softmax sur les valeurs de préférences de chacune des actions, définie par :

$\pi_t(a) = \frac{e^{H_t(a)}}{\sum_{i=1}^{k}H_t(b)}$ avec $H_t(a)$ la valeur de préférence de l'action $a$.

La mise à jour de la valeur de préférence est régie par le gradient, je vous laisser découvrir la démonstration dans l'ouvrage. Voici la règle de mise à jour :

$H_{t+1}(a) = H_t(a) + \alpha (R_t - \bar{R}_t)(1_{[a = A_t]} - \pi_t(a))$ avec $\bar{R}_t$ la moyenne des récompenses de l'action jusqu'au temps $t$ (dernier temps exclu). $\alpha$ pouvant être assimilé au *learning rate*.

Petit rappel pour réduire la gestion en mémoire et éviter d'enregistrer toutes les récompenses de toutes les actions, pour l'implémentation du calcul de $\bar{R}_t$ :

$\bar{R}_n = \frac{1}{n} \sum_{i=1}^{n} R_i$
$ = \frac{1}{n} R_n + \frac{1}{n} \times \frac{n-1}{n-1} \sum_{i=1}^{n-1} R_i$
$ = \frac{1}{n} (R_n + (n-1) \bar{R}_{n-1})$

In [ ]:
class GradientBandits(nn.Module):
    def __init__(self, alpha: float, k: int) -> None:
        super(GradientBandits, self).__init__()
        
        self.register_buffer('alpha', torch.scalar_tensor(alpha, dtype=torch.float))
        self.register_buffer('k', torch.scalar_tensor(k, dtype=torch.int))
        self.register_buffer('preferences', torch.zeros(size=(k,)))
        self.register_buffer('average_reward', torch.zeros(size=(k,), dtype=torch.float))
        self.register_buffer('average_count', torch.zeros(size=(k,), dtype=torch.int))

    def _select_action(self) -> None:
        softmax_preferences = torch.softmax(self.preferences, dim=0)
        equalities = torch.where(softmax_preferences == softmax_preferences.max())[0]

        if equalities.shape[0] > 1:
           selection = distrib.categorical.Categorical(torch.tensor([1 / equalities.shape[0] for _ in range(equalities.shape[0])])).sample().item()
           return equalities[selection]

        return torch.argmax(softmax_preferences)
    
    def _update_preferences(self, action: int, reward: float) -> None:
        choice = torch.zeros(size=(self.k,), dtype=torch.float)
        choice[action] = 1.

        self.preferences = self.preferences + self.alpha * (reward - self.average_reward) * (choice - torch.softmax(self.preferences, dim=0))
        self.average_count[action] = self.average_count[action] + 1
        self.average_reward[action] = 1 / self.average_count[action] * (reward + (self.average_count[action] - 1) * self.average_reward[action])
    
    def forward(self, kbandit: Callable) -> Tuple[torch.Tensor, torch.Tensor]:
        action = self._select_action()
        with torch.no_grad():
            reward = kbandit(action)
            self._update_preferences(action, reward)

        return (torch.scalar_tensor(action, dtype=torch.int), torch.scalar_tensor(reward, dtype=torch.float))

In [ ]:
# Gradient Bandits Algorithms
gradient_histories = []
for _ in range(2000):
    gradient_bandit = GradientBandits(0.1, 10)
    history = []
    for _ in range(1000):
        history.append(gradient_bandit(bandit_problem))
    
    gradient_histories.append(history)

gradient_histories = torch.tensor(gradient_histories)

# Visualization

Comme dans l'ouvrage les 2000 expérimentations du même problème nous permet d'obtenir le pourcentage d'apparition de la solution optimale à une étape donnée. Ce que je propose de faire dans la suite du *notebook*.

In [ ]:
# Calcul des pourcentages de décision optimale
baseline_percentages = (torch.bincount(torch.where(baseline_histories[..., 0].to(torch.int) == bandit_problem.optimal_action)[1]) / baseline_histories.shape[0])
greedy_percentages = (torch.bincount(torch.where(greedy_histories[..., 0].to(torch.int) == bandit_problem.optimal_action)[1]) / greedy_histories.shape[0])
optimistic_percentages = (torch.bincount(torch.where(optimistic_greedy_histories[..., 0].to(torch.int) == bandit_problem.optimal_action)[1]) / optimistic_greedy_histories.shape[0])
ucb_percentages = (torch.bincount(torch.where(ucb_greedy_histories[..., 0].to(torch.int) == bandit_problem.optimal_action)[1]) / ucb_greedy_histories.shape[0])
gradient_percentages = (torch.bincount(torch.where(gradient_histories[..., 0].to(torch.int) == bandit_problem.optimal_action)[1]) / gradient_histories.shape[0])

fig, ax = plt.subplots(figsize=(15, 10))

ax.plot(baseline_percentages, label='0 greedy', c='blue')
ax.plot(greedy_percentages, label='0.1 greedy', c='green')
ax.plot(optimistic_percentages, label='optimistic initial value 5', c='red')
ax.plot(ucb_percentages, label='UCB c=2', c='black')
ax.plot(gradient_percentages, label='Gradient Algorithms alpha=0.1', c='orange')

plt.legend()

ax.set_ylabel('Pourcentage apparition valeur optimale')
ax.set_xlabel('Itération')

plt.tight_layout()
plt.show()